In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
from PIL import Image
import random, time, torchprofile
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Read and Transform Data
class LoadData(Dataset):
    def __init__(self, file, root):
        self.root = root
        
        # Data tranformation
        self.transform = transforms.Compose([
            transforms.Resize(84),
            transforms.CenterCrop(84),
            transforms.RandomHorizontalFlip(1),
            transforms.RandomVerticalFlip(0.1),
            transforms.RandomPerspective(distortion_scale = 0.2, p = 0.2),
            transforms.RandomRotation(15),
            RandomChannel(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
        
        # Read the data file
        with open(file, 'r') as files:
            self.data = files.readlines()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        line = self.data[idx].strip()
        img_path, label = line.split()
        img_path = os.path.join(self.root, img_path)
        img = Image.open(img_path).convert('RGB')
        label = int(label)
        img = self.transform(img)
        return img, label

class RandomChannel:
    def __init__(self):
        self.all_channel = [
            (0, 1, 2),  # RGB
            (0, 1),     # RG
            (0, 2),     # RB
            (1, 2),     # GB
            (0,),       # R
            (1,),       # G
            (2,)        # B
        ]

    def __call__(self, img):
        channels = list(img.split())
        random_channel = random.choice(self.all_channel)
        img_merged = Image.merge('RGB', [channels[i] if i in random_channel else Image.new('L', img.size) for i in range(3)])
        return img_merged

class DynamicConv2D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride = 1, padding = 0, bias = True):
        super(DynamicConv2D, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.bias = bias

        # Define convoluion weight and bias
        self.weight = nn.Parameter(torch.randn(out_channels, in_channels, kernel_size, kernel_size))
        if bias:
            self.bias = nn.Parameter(torch.randn(out_channels))
        else:
            self.register_parameter('bias', None)

    def forward(self, x):
        
        # Adjust weight based on input channels
        channels = x.size(1)
        if channels < self.in_channels:
            weight = self.weight[:, :channels, :, :]
        else:
            weight = self.weight
        return F.conv2d(x, weight, self.bias, self.stride, self.padding)

## Train ResNet18 with DynamicConv2D

In [2]:
# Create dataset instances
train_dataset = LoadData(file = 'train.txt', root = '')
val_dataset = LoadData(file = 'val.txt', root = '')
test_dataset = LoadData(file = 'test.txt', root = '')

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 64, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False)

In [3]:
model = models.resnet18(pretrained = False)
model.conv1 = DynamicConv2D(in_channels = 3, out_channels = 64, kernel_size = 7, padding = 1)
model.fc = torch.nn.Linear(model.fc.in_features, 50)
model.to(device)

/home/re6121029/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/re6121029/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): DynamicConv2D()
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1)

In [4]:
epochs = 30
lr = 0.1
momentum = 0.9
weight_decay = 1e-4
optimizer = optim.SGD(model.parameters(), lr = lr, momentum = momentum, weight_decay = weight_decay)

def adjust_lr(optimizer, epoch):
    if epoch in [epochs * 0.5, epochs * 0.75, epochs * 0.85]:
        for p in optimizer.param_groups:
            p['lr'] *= 0.1
            lr = p['lr']
        print('Adjusted lr:' + str(lr))

def train(epoch):
    model.train()
    avg_loss = 0.
    train_acc = 0.
    adjust_lr(optimizer, epoch)
    train_loader_len = len(train_loader.dataset)
    train_loader_iter = tqdm(enumerate(train_loader), total = len(train_loader), desc = "Training Epoch #{}".format(epoch))
    
    for batch_idx, (data, label) in train_loader_iter:
        data, label = data.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, label)
        avg_loss += loss.item()
        pred = output.data.max(1, keepdim = True)[1]
        train_acc += pred.eq(label.data.view_as(pred)).cpu().sum()
        loss.backward()
        optimizer.step()
        train_loader_iter.set_postfix(loss = loss.item(), accuracy = 100. * train_acc.item() / train_loader_len)    
    print('Train Epoch: {}, Loss: {:.6f}, Accuracy: {:.2f}%'.format(epoch, avg_loss / len(train_loader), 100. * train_acc / train_loader_len))

def val(epoch):
    model.eval()
    test_loss = 0.
    correct = 0
    val_loader_iter = tqdm(val_loader, total = len(val_loader), desc = "Validation Epoch #{}".format(epoch))
    
    with torch.no_grad():
        for data, label in val_loader_iter:
            data, label = data.to(device), label.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, label, reduction = 'sum').item()
            pred = output.data.max(1, keepdim = True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
            val_loader_iter.set_postfix(loss = test_loss / len(val_loader.dataset), accuracy = 100. * correct.item() / len(val_loader.dataset))
    
    test_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print('Validation Set: Average Loss: {:.4f}, Accuracy: {:.2f}%'.format(test_loss, accuracy))
    return accuracy

best_val_acc = 0.
for i in range(epochs):
    train(i + 1)
    temp_acc = val(i + 1)
    if temp_acc > best_val_acc:
        best_val_acc = temp_acc
        torch.save(model.state_dict(), 'best_1.pt')
        print('Best Accuracy: {:.2f}%'.format(best_val_acc))
print('Final Best Accuracy: {:.2f}%'.format(best_val_acc))

Training Epoch #1: 100%|██████████| 990/990 [03:12<00:00,  5.14it/s, accuracy=4.58, loss=3.55] 


Train Epoch: 1, Loss: 3.811792, Accuracy: 4.58%


Validation Epoch #1: 100%|██████████| 8/8 [00:01<00:00,  6.56it/s, accuracy=10, loss=3.58]  


Validation Set: Average Loss: 3.5834, Accuracy: 10.00%
Best Accuracy: 10.00%


Training Epoch #2: 100%|██████████| 990/990 [03:10<00:00,  5.21it/s, accuracy=10.7, loss=2.97] 


...

Validation Set: Average Loss: 2.0369, Accuracy: 50.44%


Training Epoch #30: 100%|██████████| 990/990 [03:10<00:00,  5.20it/s, accuracy=83.6, loss=0.484]


Train Epoch: 30, Loss: 0.555541, Accuracy: 83.61%


Validation Epoch #30: 100%|██████████| 8/8 [00:01<00:00,  6.93it/s, accuracy=48.2, loss=2.11] 

Validation Set: Average Loss: 2.1070, Accuracy: 48.22%
Final Best Accuracy: 55.11%


## Load ResNet18 with DynamicConv2D

In [5]:
model = models.resnet18(pretrained = False)
model.conv1 = DynamicConv2D(in_channels = 3, out_channels = 64, kernel_size = 7, padding = 1)
model.fc = torch.nn.Linear(model.fc.in_features, 50)
model.load_state_dict(torch.load("best_1.pt"))
model.to(device)
model.eval()

ResNet(
  (conv1): DynamicConv2D()
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1)

## Evaluate by Testing Dataset

In [6]:
def test(model):
    model.eval()
    correct = 0
    test_dataset = LoadData(file = 'test.txt', root = '')
    test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False)
    test_loader_len = len(test_loader.dataset)
    test_loader_iter = tqdm(test_loader, total = len(test_loader), desc = "Testing")
    
    all_preds = []
    all_targets = []
    
    start_time = time.time()
    
    with torch.no_grad():
        for data, label in test_loader_iter:
            data, label = data.to(device), label.to(device)
            output = model(data)
            pred = output.data.max(1, keepdim = True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
            
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(label.cpu().numpy())
            
            test_loader_iter.set_postfix(accuracy = 100. * correct.item() / test_loader_len)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    accuracy = 100. * correct / test_loader_len
    

    # Calculate precision, recall, and F1-score
    precision = 100. * precision_score(all_targets, all_preds, average = 'macro')
    recall = 100. * recall_score(all_targets, all_preds, average = 'macro')
    f1 = 100. * f1_score(all_targets, all_preds, average = 'macro')
    
    # Calculate FLOPS
    flops = torchprofile.profile_macs(model, torch.randn(1, *data.shape[1:]).to(device))
    
    return accuracy.item(), precision, recall, f1, flops, elapsed_time

rgb_list = ["RGB", "RG", "RB", "GB", "R", "G", "B"]
for rgb in rgb_list:
    resnet_acc, resnet_precision, resnet_recall, resnet_f1, resnet_flops, resnet_elapsed_time = test(model)
    print(f"RGB Set: {rgb:s}, Accuracy: {resnet_acc:.2f}%, Precision: {resnet_precision:.2f}%, Recall: {resnet_recall:.2f}%, F1 Score: {resnet_f1:.2f}%, FLOPS: {resnet_flops:d}, Elapsed Time: {resnet_elapsed_time:.2f} seconds")

Testing: 100%|██████████| 8/8 [00:01<00:00,  6.80it/s, accuracy=54.2]


RGB Set: RGB, Accuracy: 54.22%, Precision: 56.11%, Recall: 54.22%, F1 Score: 53.69%, FLOPS: 926579712, Elapsed Time: 1.18 seconds


Testing: 100%|██████████| 8/8 [00:01<00:00,  6.80it/s, accuracy=53.6]


RGB Set: RG, Accuracy: 53.56%, Precision: 53.97%, Recall: 53.56%, F1 Score: 52.56%, FLOPS: 926579712, Elapsed Time: 1.18 seconds


Testing: 100%|██████████| 8/8 [00:01<00:00,  6.80it/s, accuracy=52]  


...

RGB Set: R, Accuracy: 50.00%, Precision: 51.50%, Recall: 50.00%, F1 Score: 48.99%, FLOPS: 926579712, Elapsed Time: 1.18 seconds


Testing: 100%|██████████| 8/8 [00:01<00:00,  6.80it/s, accuracy=52.7]


RGB Set: G, Accuracy: 52.67%, Precision: 54.41%, Recall: 52.67%, F1 Score: 51.93%, FLOPS: 926579712, Elapsed Time: 1.18 seconds


Testing: 100%|██████████| 8/8 [00:01<00:00,  6.65it/s, accuracy=52]  


RGB Set: B, Accuracy: 52.00%, Precision: 54.26%, Recall: 52.00%, F1 Score: 51.44%, FLOPS: 926579712, Elapsed Time: 1.20 seconds


## Train ResNet18

In [7]:
model = models.resnet18(pretrained = False)
model.fc = torch.nn.Linear(model.fc.in_features, 50)
model.to(device)

/home/re6121029/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/re6121029/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
epochs = 30
lr = 0.1
momentum = 0.9
weight_decay = 1e-4
optimizer = optim.SGD(model.parameters(), lr = lr, momentum = momentum, weight_decay = weight_decay)

def adjust_lr(optimizer, epoch):
    if epoch in [epochs * 0.5, epochs * 0.75, epochs * 0.85]:
        for p in optimizer.param_groups:
            p['lr'] *= 0.1
            lr = p['lr']
        print('Adjusted lr:' + str(lr))

def train(epoch):
    model.train()
    avg_loss = 0.
    train_acc = 0.
    adjust_lr(optimizer, epoch)
    train_loader_len = len(train_loader.dataset)
    train_loader_iter = tqdm(enumerate(train_loader), total = len(train_loader), desc = "Training Epoch #{}".format(epoch))
    
    for batch_idx, (data, label) in train_loader_iter:
        data, label = data.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, label)
        avg_loss += loss.item()
        pred = output.data.max(1, keepdim = True)[1]
        train_acc += pred.eq(label.data.view_as(pred)).cpu().sum()
        loss.backward()
        optimizer.step()
        train_loader_iter.set_postfix(loss = loss.item(), accuracy = 100. * train_acc.item() / train_loader_len)
    print('Train Epoch: {}, Loss: {:.6f}, Accuracy: {:.2f}%'.format(epoch, avg_loss / len(train_loader), 100. * train_acc / train_loader_len))

def val(epoch):
    model.eval()
    test_loss = 0.
    correct = 0
    val_loader_iter = tqdm(val_loader, total = len(val_loader), desc = "Validation Epoch #{}".format(epoch))
    
    with torch.no_grad():
        for data, label in val_loader_iter:
            data, label = data.to(device), label.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, label, reduction = 'sum').item()
            pred = output.data.max(1, keepdim = True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
            val_loader_iter.set_postfix(loss = test_loss / len(val_loader.dataset), accuracy = 100. * correct.item() / len(val_loader.dataset))
    
    test_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print('Validation Set: Average Loss: {:.4f}, Accuracy: {:.2f}%'.format(test_loss, accuracy))
    return accuracy
    
best_val_acc = 0.
for i in range(epochs):
    train(i + 1)
    temp_acc = val(i + 1)
    if temp_acc > best_val_acc:
        best_val_acc = temp_acc
        torch.save(model.state_dict(), 'resnet18_best.pt')
        print('Best Accuracy: {:.2f}%'.format(best_val_acc))
print('Final Best Accuracy: {:.2f}%'.format(best_val_acc))

Training Epoch #1: 100%|██████████| 990/990 [02:51<00:00,  5.77it/s, accuracy=4.09, loss=3.71] 


Train Epoch: 1, Loss: 3.841783, Accuracy: 4.09%


Validation Epoch #1: 100%|██████████| 8/8 [00:01<00:00,  6.75it/s, accuracy=6, loss=3.64]    


Validation Set: Average Loss: 3.6436, Accuracy: 6.00%
Best Accuracy: 6.00%


Training Epoch #2: 100%|██████████| 990/990 [02:53<00:00,  5.71it/s, accuracy=7.14, loss=3.55] 


...

Validation Set: Average Loss: 2.8161, Accuracy: 33.33%


Training Epoch #30: 100%|██████████| 990/990 [02:53<00:00,  5.69it/s, accuracy=76.7, loss=0.884]


Train Epoch: 30, Loss: 0.822482, Accuracy: 76.70%


Validation Epoch #30: 100%|██████████| 8/8 [00:01<00:00,  7.07it/s, accuracy=32.2, loss=3.03] 

Validation Set: Average Loss: 3.0281, Accuracy: 32.22%
Final Best Accuracy: 37.11%


## Load ResNet18

In [9]:
resnet18_model = models.resnet18(pretrained = False)
resnet18_model.fc = torch.nn.Linear(model.fc.in_features, 50)
resnet18_model.load_state_dict(torch.load("resnet18_best.pt"))
resnet18_model.to(device)
resnet18_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Evaluate by testing dataset

In [10]:
test_dataset = LoadData(file = 'test.txt', root = '')
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False)
def test(model):
    model.eval()
    correct = 0
    test_loader_len = len(test_loader.dataset)
    test_loader_iter = tqdm(test_loader, total = len(test_loader), desc = "Testing")
    all_preds = []
    all_targets = []
    start_time = time.time()
    
    with torch.no_grad():
        for data, label in test_loader_iter:
            data, label = data.to(device), label.to(device)
            output = model(data)
            pred = output.data.max(1, keepdim = True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(label.cpu().numpy())
            test_loader_iter.set_postfix(accuracy = 100. * correct.item() / test_loader_len)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    accuracy = 100. * correct / test_loader_len

    # Calculate precision, recall, and F1-score
    precision = 100. * precision_score(all_targets, all_preds, average = 'macro')
    recall = 100. * recall_score(all_targets, all_preds, average = 'macro')
    f1 = 100. * f1_score(all_targets, all_preds, average = 'macro')
    
    # Calculate FLOPS
    flops = torchprofile.profile_macs(model, torch.randn(1, *data.shape[1:]).to(device))
    return accuracy.item(), precision, recall, f1, flops, elapsed_time

resnet_acc, resnet_precision, resnet_recall, resnet_f1, resnet_flops, resnet_elapsed_time = test(resnet18_model)
print(f"Accuracy: {resnet_acc:.2f}%, Precision: {resnet_precision:.2f}%, Recall: {resnet_recall:.2f}%, F1 Score: {resnet_f1:.2f}%, FLOPS: {resnet_flops:d}, Elapsed Time: {resnet_elapsed_time:.2f} seconds")

Testing: 100%|██████████| 8/8 [00:01<00:00,  7.05it/s, accuracy=38.4]


Accuracy: 38.44%, Precision: 38.91%, Recall: 38.44%, F1 Score: 37.66%, FLOPS: 296456064, Elapsed Time: 1.14 seconds
